In [1]:
import os.path as pth
import fastoad.api as oad
from fastuav.utils.postprocessing.sensitivity_analysis.sensitivity_analysis import doe_fast

DATA_FOLDER_PATH = "../data"
CONFIGURATIONS_FOLDER_PATH = "../configurations"
WORK_FOLDER_PATH = "./workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "multirotor_mdo_lca.yaml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "problem_outputs_lca_mdo.xml")

oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_mdo_lca.yaml"
)

oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

In [4]:
x_dict = {
    "data:weight:arms:density": [1500, 2500, None],
    "data:structures:arms:stress:max": [70000000/2, 70000000*2, None]
         }
y_list = ["data:weight:mtow", "mission:operational:energy"]
n_samples = 10
df = doe_fast("uniform", x_dict, y_list, CONFIGURATION_FILE, n_samples)

In [5]:
df

In [15]:
import matplotlib.pyplot as plt

def plot_contour(df, x_name, y_name, z_name, levels: int = None):
    """
    Contour plot from dataframe values.
    """
    # Get data
    x = df[x_name]
    y = df[y_name]
    z = df[z_name]
    
    # Initialize plot
    fig, ax = plt.subplots()

    # Plot contour
    ax.tricontour(x, y, z, levels=levels, linewidths=0.5, colors='k')
    cntr = ax.tricontourf(x, y, z, levels=levels, cmap="RdBu_r")
    fig.colorbar(cntr, ax=ax)
    
    # Data points
    ax.plot(x, y, 'ko', ms=3)
    
    return fig

x_name = list(x_dict.keys())[0]
y_name = list(x_dict.keys())[1]
z_name = y_list[0]

fig = plot_contour(df, x_name, y_name, z_name, levels=10)